# Importing Dependencies

In [232]:
import pandas as pd

In [233]:
df=pd.read_csv('Deduplication Problem - Sample Dataset.csv')

In [234]:
df.head(8)

,ln,dob,gn,fn
0,SMITH JR,01-03-68,F,WILLIAM
1,ROTHMEYER JR,01-03-68,F,WILLIAM
2,ASBY JR,01-03-68,F,WILLIAM
3,SALTER JR,01-03-68,F,WILLIAM
4,SALTER JR,01-03-68,F,WILLIAM
5,BLAND JR,21-02-62,F,WILLIAM
6,BLAND JR,21-02-62,F,WILLIAM
7,BLAND JR,21-02-62,F,WILLIAM


In [235]:
df.describe()

,ln,dob,gn,fn
count,112,112,112,112
unique,37,24,2,28
top,MICHAELSON JR,07-10-37,M,HAROLD
freq,22,24,86,24


In [236]:
df.nunique()

ln     37
dob    24
gn      2
fn     28
dtype: int64

## Vectorizing the Text & Seperating Dates

In [237]:
from sklearn.feature_extraction.text import CountVectorizer


#from sklearn.feature_extraction.text import TfidfVectorizer


In [238]:
df['day']=pd.to_datetime(df['dob']).dt.day
df['month']=pd.to_datetime(df['dob']).dt.month
df['year']=pd.to_datetime(df['dob']).dt.year

In [239]:
df.drop('dob',axis=1,inplace=True)

In [240]:
gender_dummie=pd.get_dummies(df['gn'])

In [241]:
df=pd.concat([df,gender_dummie],axis=1)
df.drop('gn',axis=1,inplace=True)

In [242]:
from sklearn.cluster import KMeans
K=KMeans()

In [243]:
vectorizer = CountVectorizer()
Vect1=vectorizer.fit_transform(df['fn'])
vectorizer.get_feature_names()
FirstNameCol=pd.DataFrame(Vect1.toarray(),columns=vectorizer.get_feature_names())

vectorizer = CountVectorizer()
vectorizer=CountVectorizer()
Vect2=vectorizer.fit_transform(df['ln'])
SecondNameCol=pd.DataFrame(Vect2.toarray(),columns=vectorizer.get_feature_names())

In [244]:
Final=pd.concat([df,FirstNameCol,SecondNameCol],axis=1)

In [245]:
Final.drop(['ln','fn'],axis=1,inplace=True)


## Creating Manual Feature

In [246]:
from sklearn.metrics.pairwise import cosine_similarity


In [247]:
len_fn=[]
len_ln=[]

for i in range(len(df)):
    len_fn.append('fn'+str(i))
    len_ln.append('ln'+str(i))
    


In [248]:
fn_df=pd.DataFrame(3*cosine_similarity(Vect1[:], Vect1),columns=len_fn)
ln_df=pd.DataFrame(3*cosine_similarity(Vect2[:], Vect2),columns=len_ln)
Final_1=pd.concat([Final,fn_df,ln_df],axis=1)

In [249]:
2*cosine_similarity(Vect1[:], Vect1).shape

(112, 112, 112, 112)

documents = (
    "Dhruv Rawat",
    "Dhruv Raj Rawat",
    "Dhruv Raj Singh Rawat",
    "Dhruv asd"
    )
tfidf_vectorizer = TfidfVectorizer()
tfidf_matrix = tfidf_vectorizer.fit_transform(documents)
cosine_similarity(tfidf_matrix[:], tfidf_matrix).shape


# Evaluation

In [250]:
from sklearn.metrics import silhouette_score,silhouette_samples

In [257]:
score=[]
for i in range(3,100):
    Kmeans_model=KMeans(n_clusters=i).fit(Final_1)
    labels=Kmeans_model.labels_
    score.append(silhouette_score(Final_1,labels))
    print(i,silhouette_score(Final_1,labels))

3 0.3526348812135727
4 0.37301166690671467
5 0.39681237395776675
6 0.43446249548802
7 0.44768616662533994
8 0.47577875963547517
9 0.49470180436012395
10 0.522269543122207
11 0.5314913239183608
12 0.5352750465109908
13 0.5479579729436329
14 0.5525310637847781
15 0.5669481994051537
16 0.5833095798731486
17 0.5751206260595515
18 0.5994755801234506
19 0.6064774359434557
20 0.624231122073942
21 0.6413419132789666
22 0.6595190431190253
23 0.6485078490619642
24 0.6025684212272663
25 0.636934933269789
26 0.692025249849942
27 0.6302983270820557
28 0.6529754529668005
29 0.6472090584782636
30 0.6655675068902748
31 0.694077191378531
32 0.683042485633333
33 0.7147096615128419
34 0.7049372232110871
35 0.720769111603995
36 0.7147853378716794
37 0.728966346205109
38 0.7274402393111468
39 0.7352479715718481
40 0.7359163042996234
41 0.7587278385852121
42 0.7733268578503577
43 0.7728129558254994
44 0.7727985933680195
45 0.7697649805665555
46 0.7632011998981252
47 0.7626872978732668
48 0.7601562849314553


In [263]:
print (max(score))
print (score.index(max(score))+3)

0.7733268578503577
42


In [264]:
Kmeans_model=KMeans(n_clusters=42).fit(Final_1)
labels=Kmeans_model.labels_

In [265]:
labels

array([ 2,  2,  2,  2,  2,  0,  0,  0, 38, 38, 20, 30, 20,  0, 17, 17, 16,
       16, 16, 22, 34, 22, 34, 10, 12, 10, 12, 18, 26, 26, 39, 28, 28,  9,
        9, 40, 40, 40, 19, 19,  3,  3, 23, 23, 37, 11, 11, 29, 15, 15, 25,
       25, 25,  6,  6,  6,  6,  6, 21, 21, 21, 35, 36, 36, 41, 41,  8,  8,
        8,  8,  8,  8,  8,  8,  8, 27, 27, 27, 27,  7,  7, 41, 27, 27, 41,
       41,  1,  1, 31, 31, 32, 32, 32, 31, 24,  5, 24, 24, 24, 33, 13, 13,
       13, 14, 14, 14,  4,  4,  4,  4,  4,  4])

In [266]:
Score=pd.DataFrame(labels,columns=['Similarity'])

In [267]:
pd.concat([df,Score],axis=1).to_csv("clustered_4_max_score.csv",index=False)
